### Project

For this project you have 4 files containing information about persons.

The files are:
* `personal_info.csv` -   personal information such as name, gender, etc. (one row per person)
* `vehicles.csv` -   what vehicle people own (one row per person)
* `employment.csv` -   where a person is employed (one row per person)
* `update_status.csv` -   when the person's data was created and last updated

Each file contains a key, `SSN`, which **uniquely** identifies a person.

This key is present in **all** four files.

You are guaranteed that the same SSN value is present in **every** file, and that it only appears **once per file**.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.

##### Goal 1

Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

##### Goal 2

Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns.

Normally you should ensure that the SSN's match across all the files (i.e. identical row numbers in each file refer to the same SSN). However, in this case you do not need to do so as the data is already "aligned" - all the files are guaranteed to be in SSN sort order, every SSN is unique, and every SSN appears in every file.

Just make sure that in your iterable the SSN is not repeated 4 times - one time per row is enough!

##### Goal 3

Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the `last_updated` field from the `status_update` file.

##### Goal 4

Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

#### Hints

You will not be able to use a simple split approach here, as I explain in the video.

Instead you should use the `csv` module and the `reader` function.

Here's a simple example of how to use it - you will need to expand on this for your project goals, but this is a good starting point.

In [1]:
import csv

def read_file(file_name):
    with open(file_name) as f:
        rows = csv.reader(f, delimiter=',', quotechar='"')
        yield from rows
    

In [2]:
from itertools import islice

rows = read_file('personal_info.csv')
for row in islice(rows, 5):
    print(row)

['ssn', 'first_name', 'last_name', 'gender', 'language']
['100-53-9824', 'Sebastiano', 'Tester', 'Male', 'Icelandic']
['101-71-4702', 'Cayla', 'MacDonagh', 'Female', 'Lao']
['101-84-0356', 'Nomi', 'Lipprose', 'Female', 'Yiddish']
['104-22-0928', 'Justinian', 'Kunzelmann', 'Male', 'Dhivehi']


As you can see, the data is already separated into a list containing the individual fields - but of course they are all just strings.

### Good luck!

#### Goal 1
```
Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.
```

In [3]:
import csv
import itertools
from collections import namedtuple
from datetime import datetime, timezone


class FileReader:
    """
    General purpose csv file reader with basic item type validation.
    Will try to cast into `int`, `datetime` (iso format) or `str`.
    """

    def __init__(self, file_name: str = ""):
        self._file_name = file_name
        self._file_reader = self._read_file()

        # I'm making some file name asumptions, parsing file name isn't a point of this excercies
        headers = next(self._file_reader)
        self._row_factory = namedtuple(file_name.split(".")[0].title().replace("_", ""), headers)
    
    @property
    def file_name(self):
        return self._file_name

    @property
    def fields(self):
        return self._row_factory._fields

    def __iter__(self):
        return self
        
    def __next__(self):
        row = next(self._file_reader)
        row = self._cast_row_types(row)   
        return self._row_factory(*row)

    def _read_file(self):
        with open(self.file_name) as f:
            rows = csv.reader(f, delimiter=',', quotechar='"')
            yield from rows

    def _cast_row_types(self, row: str):
        row_updated_types = []
        for el in row:
            try:
                el: datetime.datetime = datetime.fromisoformat(el)
                row_updated_types.append(el)
                continue
            except ValueError:
                pass

            if el.isdigit():
                row_updated_types.append(int(el))
            else:
                row_updated_types.append(str(el))

        return row_updated_types
        


PersonalInfoReader = FileReader("personal_info.csv")
EmploymentInfoReader = FileReader("employment.csv")
VehicleInfoReader = FileReader("vehicles.csv")
UpdateStatusInfoReader = FileReader("update_status.csv")


In [4]:
from itertools import chain, islice


file_readers = chain(
    islice(PersonalInfoReader, 0, 5), 
    islice(EmploymentInfoReader, 0, 5), 
    islice(VehicleInfoReader, 0, 5), 
    islice(UpdateStatusInfoReader, 0, 5),
)

for fr in file_readers:
    print(fr)
    print()


PersonalInfo(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')

PersonalInfo(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')

PersonalInfo(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')

PersonalInfo(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')

PersonalInfo(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')

Employment(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824')

Employment(employer='Nicolas and Sons', department='Sales', employee_id='41-6841359', ssn='101-71-4702')

Employment(employer='Connelly Group', department='Research and Development', employee_id='98-7952860', ssn='101-84-0356')

Employment(employer='Upton LLC', department='Marketing', employee_id='56-981755

#### Goal 2
Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns.

Normally you should ensure that the SSN's match across all the files (i.e. identical row numbers in each file refer to the same SSN). However, in this case you do not need to do so as the data is already "aligned" - all the files are guaranteed to be in SSN sort order, every SSN is unique, and every SSN appears in every file.

Just make sure that in your iterable the SSN is not repeated 4 times - one time per row is enough!

In [5]:
PersonalInfoReader = FileReader("personal_info.csv")
EmploymentInfoReader = FileReader("employment.csv")
VehicleInfoReader = FileReader("vehicles.csv")
UpdateStatusInfoReader = FileReader("update_status.csv")

combined_data_row = set(field for reader in (PersonalInfoReader, EmploymentInfoReader, VehicleInfoReader, UpdateStatusInfoReader) for field in reader.fields)
DataRow = namedtuple("DataRow", ",".join(combined_data_row))


def person_data():
    for _ in range(5):
        # data is presorted so it's fine - IDs (SSN) will match on order alone
        row_data = {
            **(next(PersonalInfoReader))._asdict(),
            **(next(EmploymentInfoReader))._asdict(),
            **(next(VehicleInfoReader))._asdict(),
            **(next(UpdateStatusInfoReader))._asdict(),
        }
        yield DataRow(**row_data)


for d in person_data():
    print(d)
    print()


DataRow(employee_id='29-0890771', vehicle_model='Bravada', first_name='Sebastiano', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42, tzinfo=datetime.timezone.utc), ssn='100-53-9824', department='Research and Development', created=datetime.datetime(2016, 1, 24, 21, 19, 30, tzinfo=datetime.timezone.utc), employer='Stiedemann-Bailey', vehicle_make='Oldsmobile', model_year=1993, gender='Male', last_name='Tester', language='Icelandic')

DataRow(employee_id='41-6841359', vehicle_model='Mustang', first_name='Cayla', last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17, tzinfo=datetime.timezone.utc), ssn='101-71-4702', department='Sales', created=datetime.datetime(2016, 1, 27, 4, 32, 57, tzinfo=datetime.timezone.utc), employer='Nicolas and Sons', vehicle_make='Ford', model_year=1997, gender='Female', last_name='MacDonagh', language='Lao')

DataRow(employee_id='98-7952860', vehicle_model='Yukon', first_name='Nomi', last_updated=datetime.datetime(2017, 10, 4, 11, 21, 30, tzinfo=datetime.

#### Goal 3
Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the last_updated field from the status_update file.

In [6]:

def recently_updated_person_data():
    PersonalInfoReader = FileReader("personal_info.csv")
    EmploymentInfoReader = FileReader("employment.csv")
    VehicleInfoReader = FileReader("vehicles.csv")
    UpdateStatusInfoReader = FileReader("update_status.csv")
    combined_data_row = set(
        field 
        for reader in (PersonalInfoReader, EmploymentInfoReader, VehicleInfoReader, UpdateStatusInfoReader) 
        for field in reader.fields
    )
    DataRow = namedtuple("DataRow", ",".join(combined_data_row))

    cut_out_date = datetime(2017, 3, 1, tzinfo=timezone.utc)
    while True:
        update_status = next(UpdateStatusInfoReader, None)
        if not update_status:
            return 

        # takewhile / dropwhile could be used
        while update_status.last_updated <= cut_out_date:
            next(PersonalInfoReader)
            next(EmploymentInfoReader)
            next(VehicleInfoReader)
            update_status = next(UpdateStatusInfoReader, None)
            if not update_status:
                return 

        row_data = {
            **(next(PersonalInfoReader))._asdict(),
            **(next(EmploymentInfoReader))._asdict(),
            **(next(VehicleInfoReader))._asdict(),
            **(update_status)._asdict(),
        }
        yield DataRow(**row_data)

for d in itertools.islice(recently_updated_person_data(), 150, 155):
    print(d)
    print()


DataRow(employee_id='82-2230506', vehicle_model='Sunfire', first_name='Clay', last_updated=datetime.datetime(2018, 3, 26, 6, 15, 57, tzinfo=datetime.timezone.utc), ssn='230-48-0684', department='Product Management', created=datetime.datetime(2016, 12, 29, 6, 0, 3, tzinfo=datetime.timezone.utc), employer='Fahey Group', vehicle_make='Pontiac', model_year=1999, gender='Male', last_name='Peaurt', language='Dhivehi')

DataRow(employee_id='99-0856844', vehicle_model='CC', first_name='Markus', last_updated=datetime.datetime(2017, 8, 10, 19, 34, 42, tzinfo=datetime.timezone.utc), ssn='230-99-1485', department='Marketing', created=datetime.datetime(2016, 9, 19, 22, 26, 22, tzinfo=datetime.timezone.utc), employer='Hodkiewicz-Murray', vehicle_make='Volkswagen', model_year=2009, gender='Male', last_name='Revill', language='Moldovan')

DataRow(employee_id='68-3132488', vehicle_model='Sonoma Club Coupe', first_name='Gisele', last_updated=datetime.datetime(2017, 4, 7, 22, 50, 50, tzinfo=datetime.time

#### Goal 4
Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

In [7]:
import contextlib
from itertools import groupby

grouping_key = lambda data_row: (data_row.gender, data_row.vehicle_make)

# before grouping, we need to sort the data so duplicated groups are not created
recently_updated = sorted(recently_updated_person_data(), key=grouping_key)
groups = groupby(recently_updated, key=grouping_key)

genders_and_makes = []
for gender_and_make, make_list in groups:
    genders_and_makes.append((gender_and_make, len(list(make_list))))

for row in sorted(genders_and_makes, key=lambda x: x[1], reverse=True):
    print(row)


(('Female', 'Chevrolet'), 42)
(('Female', 'Ford'), 42)
(('Male', 'Ford'), 40)
(('Male', 'Chevrolet'), 30)
(('Male', 'GMC'), 28)
(('Male', 'Mitsubishi'), 28)
(('Female', 'GMC'), 22)
(('Female', 'Mitsubishi'), 22)
(('Male', 'Dodge'), 22)
(('Male', 'Toyota'), 21)
(('Female', 'Toyota'), 20)
(('Male', 'Mercedes-Benz'), 19)
(('Female', 'Dodge'), 17)
(('Female', 'Mercedes-Benz'), 17)
(('Male', 'Volkswagen'), 16)
(('Female', 'Lexus'), 15)
(('Female', 'Pontiac'), 14)
(('Male', 'Audi'), 14)
(('Female', 'Audi'), 13)
(('Female', 'Mazda'), 13)
(('Female', 'Volvo'), 13)
(('Male', 'Buick'), 13)
(('Male', 'Mazda'), 13)
(('Female', 'BMW'), 12)
(('Female', 'Nissan'), 12)
(('Female', 'Suzuki'), 12)
(('Male', 'BMW'), 12)
(('Female', 'Buick'), 11)
(('Male', 'Mercury'), 11)
(('Male', 'Pontiac'), 11)
(('Female', 'Volkswagen'), 10)
(('Male', 'Volvo'), 10)
(('Female', 'Acura'), 9)
(('Female', 'Infiniti'), 9)
(('Female', 'Kia'), 9)
(('Male', 'Cadillac'), 9)
(('Male', 'Honda'), 9)
(('Female', 'Honda'), 8)
(('Fem